In [5]:
import polars as pl
from sklearn.neighbors import NearestNeighbors  # k-NN

k_in_knn = 5  # k-NN における k

dataset = pl.read_csv("../test_data/resin.csv")
x_prediction = pl.read_csv("../test_data/resin_prediction.csv")

# データ分割
y = dataset.get_column("property")  # 目的変数
x = dataset.drop(y.name)  # 説明変数

# 標準偏差が 0 の特徴量の削除
deleting_variables = [
    col for col, std in x.std().row(0, named=True).items() if std in [0, None]
]
x = x.drop(deleting_variables)
x_prediction = x_prediction.drop(deleting_variables)

# オートスケーリング
autoscaled_x = x.select((pl.all() - pl.all().mean()) / pl.all().std())
autoscaled_x_prediction = (
    x_prediction - x.mean().select(pl.all().repeat_by(x_prediction.height).explode())
) / x.std().select(pl.all().repeat_by(x_prediction.height).explode())

# k-NN による AD
ad_model = NearestNeighbors(n_neighbors=k_in_knn, metric="euclidean")  # AD モデルの宣言
ad_model.fit(
    autoscaled_x
)  # k-NN による AD では、トレーニングデータの x を model_ad に格納することに対応

# サンプルごとの k 最近傍サンプルとの距離に加えて、k 最近傍サンプルのインデックス番号も一緒に出力されるため、出力用の変数を 2 つに
# トレーニングデータでは k 最近傍サンプルの中に自分も含まれ、自分との距離の 0 を除いた距離を考える必要があるため、k_in_knn + 1 個と設定
knn_distance_train, knn_index_train = ad_model.kneighbors(
    autoscaled_x, n_neighbors=k_in_knn + 1
)
test = pl.DataFrame(knn_distance_train).drop(pl.first())
mean_of_knn_distance_train = (
    # 自分以外の k_in_knn 個の距離の平均
    pl.DataFrame(knn_distance_train)
    .drop(pl.first())
    .mean_horizontal()
    .alias("mean_of_knn_distance")
)
mean_of_knn_distance_train.to_frame().write_csv(
    "../output/mean_of_knn_distance_train.csv"
)

# AD 内となるトレーニングデータの割合。ADのしきい値を決めるときに使用
rate_of_training_samples_inside_ad = 0.96
# トレーニングデータのサンプルの rate_of_training_samples_inside_ad * 100 % が含まれるようにしきい値を設定
ad_threshold: float = mean_of_knn_distance_train.sort().item(
    # サンプル数の96%の位置の値を閾値とする
    round(autoscaled_x.height * rate_of_training_samples_inside_ad) - 1
)

# トレーニングデータに対して、AD の中か外かを判定
inside_ad_flag_train = mean_of_knn_distance_train.le(ad_threshold).alias(
    "inside_ad_flag"
)
inside_ad_flag_train.to_frame().write_csv("../output/inside_ad_flag_train_knn.csv")

# 予測用データに対する k-NN 距離の計算
knn_distance_prediction, knn_index_prediction = ad_model.kneighbors(
    autoscaled_x_prediction
)
# k_in_knn 個の距離の平均
mean_of_knn_distance_prediction = (
    pl.DataFrame(knn_distance_prediction)
    .mean_horizontal()
    .alias("mean_of_knn_distance")
)
mean_of_knn_distance_prediction.to_frame().write_csv(
    "../output/mean_of_knn_distance_prediction.csv"
)

# 予測用データに対して、AD の中か外かを判定
inside_ad_flag_prediction = mean_of_knn_distance_prediction.le(ad_threshold)
inside_ad_flag_prediction.to_frame().write_csv(
    "../output/inside_ad_flag_prediction_knn.csv"
)